(results)=
# Results

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
plt.style.use('paper')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf')

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate

from d2e2f.pipelines.trip_statistics import clean_statistics
from scipy.stats import norm
from myst_nb import glue
#plt.style.use('presentation')
from IPython.display import display, Markdown, Latex
import sympy as sp
from d2e2f.visualization.visualize import plot_map, plot_trips
import statsmodels.api as sm
import geopandas as gp
from d2e2f.reporting import pop_index, reload_kedro
import yaml
catalog = reload_kedro()

In [ ]:
with open('metadata.yml',mode='r') as file:
    metadata = yaml.safe_load(file)
ship=metadata['ship']
shipname = metadata['shipname']

df = catalog.load(f'{ship}.data_with_trip_columns')
df['time'] = df.index
loader_raw = catalog.load(f'{ship}.raw_data')
df_raw = loader_raw[list(loader_raw.keys())[-1]]()
df_raw.index = pd.to_datetime(df_raw['Timestamp'])
dt = pd.to_datetime(df_raw['Timestamp']).diff().mean()
t_start = df.iloc[0].name
t_end = df.iloc[-1].name

trip_statistics = catalog.load(f'{ship}.trip_statistics')
trip_statistics_clean = catalog.load(f'{ship}.trip_statistics_clean')
trip_statistics_clean['trip_no'] = trip_statistics_clean.index
trip_statistics_clean.index = pd.to_datetime(trip_statistics_clean['start_time'])
trip_statistics_clean.index.name = 'time'

harbours = catalog.load(f'params:{ship}.harbours')
min_time = catalog.load(f'params:{ship}.min_time')
max_time = catalog.load(f'params:{ship}.max_time')
min_distance = catalog.load(f'params:{ship}.min_distance')
min_start_speed = catalog.load(f'params:{ship}.min_start_speed')
max_time_diff = catalog.load(f"params:{ship}.max_time_diff")

In [ ]:
with open('metadata.yml',mode='r') as file:
    metadata = yaml.safe_load(file)
ship=metadata['ship']
shipname = metadata['shipname']

df = catalog.load(f'{ship}.data_with_trip_columns')
df['time'] = df.index
loader_raw = catalog.load(f'{ship}.raw_data')
df_raw = loader_raw[list(loader_raw.keys())[-1]]()
df_raw.index = pd.to_datetime(df_raw['Timestamp'])
dt = pd.to_datetime(df_raw['Timestamp']).diff().mean()
t_start = df.iloc[0].name
t_end = df.iloc[-1].name

trip_statistics = catalog.load(f'{ship}.trip_statistics')
trip_statistics_clean = catalog.load(f'{ship}.trip_statistics_clean')
trip_statistics_clean['trip_no'] = trip_statistics_clean.index
trip_statistics_clean.index = pd.to_datetime(trip_statistics_clean['start_time'])
trip_statistics_clean.index.name = 'time'

harbours = catalog.load(f'params:{ship}.harbours')
min_time = catalog.load(f'params:{ship}.min_time')
max_time = catalog.load(f'params:{ship}.max_time')
min_distance = catalog.load(f'params:{ship}.min_distance')
min_start_speed = catalog.load(f'params:{ship}.min_start_speed')
max_time_diff = catalog.load(f"params:{ship}.max_time_diff")

In [ ]:
glue('harbour1', list(harbours.keys())[0])
glue('harbour2', list(harbours.keys())[1])
glue('min_time', min_time)
glue('max_time', max_time)
glue('min_distance', min_distance)
glue('min_start_speed', min_start_speed)
glue('shipname', shipname)
glue('max_time_diff', max_time_diff)
glue('dt',dt)
glue('t_start',str(t_start.date()))
glue('t_end',str(t_end.date()))
glue('no_samples',np.round(len(df)/(10**6), decimals=1))
glue('direction1',trip_statistics_clean['trip_direction'].unique()[0])
glue('direction2',trip_statistics_clean['trip_direction'].unique()[1])
glue('no_trips',len(trip_statistics_clean))

## Trips

In [ ]:
interesting = ['latitude',
               'longitude',
               'sog',
               'cog',
               'consumption',
               'trip_time',
               'distance',
              ]

for i,(direction, trips_) in enumerate(trip_statistics_clean.groupby(by='trip_direction', sort=False)):    
    df_ = trips_[interesting].describe()
    
    #df_['Statistic'] = df_.index
    #columns = list(df_.columns)
    #columns.remove('Statistic')
    #columns = ['Statistic'] + columns
    
    decimals = {
        'latitude' : 3,
        'longitude' : 3,
        'sog':2,
        'consumption':2,
    }
    for key in df_.columns:
        if not key in decimals:
            decimals[key] = 1
            
    table = df_.drop(index='count').round(decimals).transpose()
    table.index.name='Signal'
    glue(f'trips_describe_{i}',pop_index(table))


In [ ]:
decimals = {
'distance':0,
}

def format(value, decimal):
    value = value.round(decimal)
    if decimal==0:
        value=int(value)
    return value

for key in interesting:
    
    decimal=decimals.get(key,1)
        
    glue(f'{key}_min',format(trip_statistics_clean[key].min(), decimal=decimal), display=False)
    glue(f'{key}_max',format(trip_statistics_clean[key].max(), decimal=decimal), display=False)
    glue(f'{key}_std',format(trip_statistics_clean[key].std(), decimal=decimal), display=False)
    
    

Statistics for the {glue:}`no_trips` trips are summarized in {numref}`tab:trips_describe_0` for the {glue:text}`direction1` direction and in {numref}`tab:trips_describe_1` for the {glue:text}`direction2` direction.

```{glue:figure} trips_describe_0
:figwidth: 300px
:name: "tab:trips_describe_0"

Trips {glue:text}`direction1`
```

```{glue:figure} trips_describe_1
:figwidth: 300px
:name: "tab:trips_describe_1"

Trips {glue:text}`direction2`
```

Latitude, longitude, speed over ground (sog) and course over ground (cog) are expressed as the mean values for each trip. Consumption, trip_time and distance are expressed as the total fuel consumption (in liters), total trip duration (in seconds) and total distance (in meters).

The statistics display a very consistent way to operate this ship; the speed over ground (sog) varies between {glue:}`sog_min` and {glue:}`sog_max` m/s with a standard deviation of {glue:}`sog_std` m/s; the distance varies between {glue:}`distance_min` and {glue:}`distance_max` meters with a standard deviation of {glue:}`distance_std` meters.

The total fuel consumption varies between {glue:}`consumption_min` and {glue:}`consumption_max` liters per trip with a standard deviation of {glue:}`consumption_std` liters.

The mean consumption is a bit higher for trips in the {glue:text}`direction2` direction, which can also be seen in {numref}`fig:fuel_hist`.

```{glue:figure} fig:fuel_hist
:name: "fig:fuel_hist"

Histogram over the fuel consumption per trip in the two directions.
```

In [ ]:
fig,ax=plt.subplots()
trip_statistics_clean['trip direction'] = trip_statistics_clean['trip_direction']
sns.histplot(trip_statistics_clean, x='consumption', hue='trip direction', ax=ax);

fig_name = 'fig:fuel_hist'
glue(fig_name, fig, display=False)

## Crew training

In [ ]:
period_starts = metadata.get('period_starts',{'start':'start'})
period_starts = {key: t_start.date() if value=='start' else value for key,value in period_starts.items()}
ends = list(period_starts.values())[1:] + [t_end.date()]

for (period,start),end in zip(period_starts.items(),ends):
        
    glue(period,start, display=False)
    
    trip_statistics_clean.loc[str(start):str(end),'period'] = period

    
    
#trip_statistics_clean.loc[utbildning:lowseason,'period'] = r'After training high season'
#trip_statistics_clean.loc[lowseason:,'period'] = r'After training low season'

Crew training in eco driving to reduce the fuel consumption of the operation of {glue:text}`shipname` was carried out {glue:text}`training`. Fuel consumption for trips before and after this training can be seen in {numref}`fig:pre_after`. The average fuel consumption per trip is very similar before and after training as seen in {numref}`fig:pre_after`.

```{glue:figure} fig:pre_after
:name: "fig:pre_after"

Fuel consumption for all trips before and after crew training.
```

In [ ]:
fig,ax=plt.subplots()

styles = ['k.','g.','b.']
for style, (period, df_period) in zip(styles,trip_statistics_clean.groupby(by='period', sort=False)):
    plot = df_period.plot(y='consumption', label=period, ax=ax, style=style, alpha=0.5)
    xs = [df_period.index[0],df_period.index[-1]]
    ys = np.array([1,1])*df_period['consumption'].mean()
    ax.plot(xs, ys, '-', label=f"Average {period.lower()}", color=plot.lines[-1].get_color(), zorder=10)
    
    
    ax.set_ylabel('Fuel consumption per trip [liters]')

#get handles and labels
handles, labels = plt.gca().get_legend_handles_labels()

#specify order of items in legend
order = list(np.arange(0,len(handles),2)) + list(np.arange(1,len(handles),2))
    
ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order], ncol=2, loc='upper center');
glue('fig:pre_after',fig, display=False)

In [ ]:
df_compare = trip_statistics_clean.groupby(by='period', sort=False)[interesting].mean()

decimals = {
    'latitude 	' : 3,
    'longitude' : 3,
}
for key in df_compare.columns:
    if not key in decimals:
        decimals[key] = 1

table = df_compare.round(decimals=decimals)
glue('tab:savings',pop_index(table))

In [ ]:
import matplotlib.ticker as mtick
df_ = df_compare[interesting]
fig,ax=plt.subplots()
df_ = df_.divide(df_.iloc[0])*100
df_.transpose().plot(kind='bar', ax=ax)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3)

ax.set_ylim(df_.min().min()-0.05*np.abs(df_.min().min()),df_.max().max()+0.03*np.abs(df_.max().max()))
ax.grid(True)
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
glue('fig:savings',fig, display=False)

Mean values for the periods are shown in {numref}`tab:savings` and {numref}`fig:savings`.

```{glue:figure} tab:savings
:name: "tab:savings"

Mean values for trips for each period.
```
There is a small tendency that the fuel consumption is larger during the low season.
```{glue:figure} fig:savings
:name: "fig:savings"

Mean values for trips for each period as percentage of values before training.
```

## Trip analysis

The average trip can be studied by calculating the mean speed and consumption for each time stamp along the trips. The average speed over ground (sog) for the trip time stamps is displayed for each period in {numref}`fig:mean_speed`. The speed is lower in the beginning and higher in the end of the trips {glue:text}`direction1` at the low season which is producing a higher fuel consumption as displayed in {numref}`fig:mean_consumption`.

```{glue:figure} fig:mean_speed
:name: "fig:mean_speed"

Speed over ground during average trips.

```

```{glue:figure} fig:mean_consumption
:name: "fig:mean_consumption"

Fuel consumption during average trips.

```


In [ ]:
fig,axes=plt.subplots(nrows=2)

for ax, (trip_direction, df_) in zip(axes,trip_statistics_clean.groupby(by=['trip_direction'], sort=False)):
    
    for period, df__ in df_.groupby(by=['period'], sort=False):
        
        mask = df['trip_no'].isin(df__['trip_no'].unique())
        df.loc[mask].groupby('trip_time')['sog'].mean().plot(label=period, ax=ax)
        ax.set_xlim(0,1450)
        ax.set_xlabel('')
        ax.set_ylabel('sog [m/s]')
        ax.set_title(trip_direction)
        ax.legend()
        
ax.set_xlabel('trip time [s]');
ax.get_legend().set_visible(False)
glue('fig:mean_speed',fig, display=False) 

In [ ]:
fig,axes=plt.subplots(nrows=2)

for ax, (trip_direction, df_) in zip(axes,trip_statistics_clean.groupby(by=['trip_direction'], sort=False)):
    
    for period, df__ in df_.groupby(by=['period'], sort=False):
        
        mask = df['trip_no'].isin(df__['trip_no'].unique())
        df.loc[mask].groupby('trip_time')['consumption'].mean().plot(label=period, ax=ax)
        ax.set_xlim(0,1450)
        ax.set_xlabel('')
        ax.set_ylabel('Fuel consumption')
        ax.set_title(trip_direction)
        ax.legend()
        
ax.set_xlabel('trip time [s]');
ax.get_legend().set_visible(False)
glue('fig:mean_consumption',fig, display=False) 